In [ ]:
import os
import sys

sys.path.append('src/')

import logging
import sys

import hydra
import mlflow
import numpy as np
import pandas as pd
import torch
from joblib import Memory
from omegaconf import DictConfig, OmegaConf

from evaluators import torchFastTextEvaluator
from framework_classes import (
    DATA_GETTER,
    DATASETS,
    LOSSES,
    MODELS,
    MODULES,
    OPTIMIZERS,
    PREPROCESSORS,
    SCHEDULERS,
    TOKENIZERS,
    TRAINERS,
)
from utils.data import get_df_naf, get_file_system, get_processed_data, get_test_data, get_Y
from utils.mappings import mappings
from utils.mlflow import create_or_restore_experiment

%load_ext autoreload
%autoreload 2

In [ ]:
revision = "NAF2008"
model_class = "torchFastText"
start_month = 1
start_year = 2018
text_feature = "libelle"
textual_features_1 = "NAT_LIB"
textual_features_2 = "AGRI"
categorical_features_1 = "TYP"
categorical_features_2 = "NAT"
categorical_features_3 = "SRF"
categorical_features_4 = "CJ"
categorical_features_5 = "CRT"

In [ ]:
cfg_dict = {"data": 
                {"sirene":"sirene_4", 
                "start_month": start_month, 
                "start_year": start_year, 
                "revision": revision,
                "text_feature": text_feature,
                "textual_features" : [textual_features_1, textual_features_2],
                "categorical_features" : [categorical_features_1, categorical_features_2, categorical_features_3, categorical_features_4, categorical_features_5],}, 
                
            "model":{"name": "torchFastText",
                    "preprocessor": "PyTorch",}}
cfg_dict_data = cfg_dict["data"]

In [ ]:
df_s3, df_s4 = DATA_GETTER[cfg_dict_data["sirene"]](**cfg_dict_data)
df_s4 = df_s4.sample(frac= 0.001, random_state=42)

In [ ]:
preprocessor = PREPROCESSORS[cfg_dict["model"]["preprocessor"]](cfg_dict)
preprocessor

In [ ]:
df_naf = get_df_naf(revision=cfg_dict_data["revision"])
Y = get_Y(revision=revision)
df_train_s4, df_val_s4, df_test = preprocessor.preprocess(
            df=df_s4,
            df_naf=df_naf,
            y=Y,
            text_feature=cfg_dict_data["text_feature"],
            textual_features=cfg_dict_data["textual_features"],
            categorical_features=cfg_dict_data["categorical_features"],
            test_size=0.1,
        )


In [ ]:

df_test_ls= get_test_data(**cfg_dict["data"], y=Y)


In [ ]:

pd.concat(preprocessor.preprocess(df_test_ls,
            df_naf=df_naf,
            y=Y,
            text_feature=cfg_dict_data["text_feature"],
            textual_features=cfg_dict_data["textual_features"],
            categorical_features=cfg_dict_data["categorical_features"],
            test_size=0.1,), axis=0)

In [ ]:
_, _, df_test = get_processed_data()


In [ ]:
df_test

In [ ]:
import mlflow

mlflow.set_tracking_uri("https://projet-ape-mlflow.user.lab.sspcloud.fr/")
mlflow.set_experiment('model_comparison_s4')
logged_model = 'runs:/45afc22a961a4cdcb282aad93693326d/model'

# Load model as a PyFuncModel.
module = mlflow.pytorch.load_model(logged_model)

In [ ]:
module.__dict__["_modules"]

In [ ]:
evaluator = torchFastTextEvaluator(module)

In [ ]:
evaluator.scheduler_interval